#Feature Extraction with Data Augmentation & Fine Tuning the Model 



## Mounting Google Drive


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Preprocessing of Data for Training

*   X_train: Colored images stored as input tensor of shape (width, height, channel)
*   Y_train: Labels are 5 face shapes output as vector [round, oval, square, oblong, heart]

# Major Steps to generate the input to CNN - using ImageDataGenerator


1.   Read the picture files.
2.   Decode the JPEG content to RGB grids of pixels.
3.   Convert these into floating-point tensors.
4.   Rescale the pixel values (between 0 and 255) to the [0, 1] interval 









In [2]:
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

## Insert the location where you put the image dataset
base_dir = r'gdrive/My Drive/face_shape_one'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

rescale_factor = 1./255

train_datagen = ImageDataGenerator(rescale= rescale_factor,
                                  width_shift_range= 40,
                                  height_shift_range= 0.2,
                                  shear_range= 0.2,
                                  zoom_range= 0.2,
                                  horizontal_flip= True,
                                  fill_mode= 'nearest')


Using TensorFlow backend.


## Data Augmentation

In [0]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

# An image chosen from the dataset
img_path = 'gdrive/My Drive/face_shape_one/train/heart/img_no_87.jpg'

# Reads the image and resizes it
img = image.load_img(img_path, target_size=(150, 150))

# Converts to Numpy array with shape (150, 150, 3)
x = image.img_to_array(img)

# Reshapes into (1, 150, 150, 3)
x = x.reshape((1,) + x.shape)

i = 0
for batch in train_datagen.flow(x, batch_size=1):
  plt.figure(i)
  imgplot = plt.imshow(image.array_to_img(batch[0]))
  i += 1
  if i%5 == 0:
    break

plt.show()

# Preprocessing of Validation & Testing Dataset

In [0]:
test_datagen = ImageDataGenerator(rescale= rescale_factor)

target_width = 150
target_height = 150

train_sample_size = 400
train_batch_size = 40

valid_sample_size = 50
valid_batch_size = 10

test_sample_size = 50
test_batch_size = 50

train_generator = train_datagen.flow_from_directory(
                                directory= train_dir, 
                                target_size= (target_width, target_height),
                                batch_size= train_batch_size, 
                                class_mode= 'categorical')


validation_generator = test_datagen.flow_from_directory(
                                directory= validation_dir, 
                                target_size= (target_width, target_height), 
                                batch_size= valid_batch_size,
                                class_mode= 'categorical')

test_generator = test_datagen.flow_from_directory(
                                directory= test_dir, 
                                target_size= (target_width, target_height), 
                                batch_size= test_batch_size,
                                class_mode= 'categorical')

##Loading MobileNetV2 model

In [0]:
from keras.applications import MobileNetV2
conv_base = MobileNetV2(weights= 'imagenet', include_top= False, input_shape= (150, 150, 3))

##Architecture of MobileNetV2

In [21]:
conv_base.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

##Building the model

* 53 layers in MobileNetV2 convolutional base
* 1 layer to flatten the input + 2 Dropout layers
* 2 Fully Connected layers - 512 & 256 nodes 

### Name of major layers in MobileNetV2:
* Depthwise Convolutional Layer
* Batch Normalization
* ReLU 

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation= 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation= 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

##Freeze Conv Base before training


*   Prevents weight from getting updated



In [0]:
conv_base.trainable = False

##Compile the learning environment

In [0]:
from keras import optimizers
model.compile(loss= 'categorical_crossentropy', 
              optimizer= optimizers.RMSprop(learning_rate=2e-5),
              metrics= ['acc'])

##Training end-to-end on Frozen layers

Number of iterations: 45

In [0]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size

history = model.fit_generator(
                    generator= train_generator,
                    steps_per_epoch= STEP_SIZE_TRAIN, 
                    epochs= 45, 
                    validation_data= validation_generator, 
                    validation_steps= STEP_SIZE_VALID)

**Plotting the result**

In [0]:
import matplotlib.pyplot as plt

# Plotting the results with smooth curve
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
       
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
           
        else:
            smoothed_points.append(point)
    return smoothed_points

def plot_result(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(1, len(acc)+1)
  plt.plot(epochs, smooth_curve(acc), 'bo', label='Smoothed training acc')
  plt.plot(epochs, smooth_curve(val_acc), 'b', label='Smoothed validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, smooth_curve(loss), 'bo', label='Smoothed training loss')
  plt.plot(epochs, smooth_curve(val_loss), 'b', label='Smoothed validation loss')
  plt.title('Training and validation loss') 
  plt.legend()
  plt.show()

plot_result(history)

#Fine-Tuning the model 

Goal: Slightly adjusts the more abstract representations of the model being reused, to make them more relevant to the problem at hand. 


In [0]:
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block_15_expand':
    set_trainable = True

  if set_trainable:
    layer.trainable = True

  else:
    layer.trainable = False

##Recompile the model

In [0]:
model.compile(loss= 'categorical_crossentropy', 
              optimizer= optimizers.RMSprop(learning_rate=1e-5),
              metrics= ['acc'])

##Retrain the model

Number of iterations: 50

In [0]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size

history = model.fit_generator(
                    generator= train_generator,
                    steps_per_epoch= STEP_SIZE_TRAIN, 
                    epochs= 50, 
                    validation_data= validation_generator, 
                    validation_steps= STEP_SIZE_VALID)

##Plotting the results

In [0]:
plot_result(history)

**Evaluating the model on test dataset (new X)**

In [0]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

test_loss, test_acc = model.evaluate_generator(test_generator, STEP_SIZE_TEST)


In [0]:
print('Accuracy on the dataset: ', test_acc*100, '%')

In [0]:
model.save('mobileNetv2_aug_face_shape_one.h5')

#Final Thoughts on Machine Learning model

Two approaches of pretrianed convent
*   Feature Extraction with No Augmentation
*   Feature Extraction with Augmentation + Fine Tuning




---


Attempts to improve model accuracy & overfitting:
1.   Increase the number of images in the dataset (9x training images)
2.   Data Augmentation 
3.   Reduce the learning capacity of the model
4.   L1 and L2 regularization
5.   Adding Dropout Layers

---
Dataset:


*   Variation in dataset
*   Room for subjectivity

---

What's next?
1. Cross-Validation
2. Refine or build a CNN from scratch


